Ideas:

* Add FFT channels to CNN (Tried, does not make much difference)
* Finetune CNN (with SGD slow learning rate)
* 5-fold CNN
* Extract Features from CNN (before FC) and do XGB
* TTA (tried, made it better)
* More augmenting, additional 45, 135, 315 degrees
* More augmenting, random rotations and flips
* Predict test data and train with test
* Train on all of the training data (no train-val split)
* Try a different combination of combine predictions
* Fine-tune on pre-trained models (Get rid of some top layers because input size is small)

In [1]:
import numpy as np
import pandas as pd
from skimage import transform
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras import losses, optimizers, callbacks
import xgboost as xgb
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate, train_test_split, StratifiedKFold, KFold
from scipy import fftpack
import matplotlib.pyplot as plt
%matplotlib inline
RANDOM_SEED = 43
np.random.seed(RANDOM_SEED)

Using TensorFlow backend.
/home/can/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Create Dataset 

In [2]:
# Train
df = pd.read_json('./data/train.json')
X, y = [], []
for im_band1, im_band2, label in zip(df['band_1'], df['band_2'], df['is_iceberg']):
    im_band1 = np.array(im_band1).reshape(75, 75, 1)
    im_band2 = np.array(im_band2).reshape(75, 75, 1)    
    # Preprocess
    # - Zero mean
    im_band1 -= np.mean(im_band1)
    im_band2 -= np.mean(im_band2)
    # - Normalize
    im_band1 /= np.std(im_band1)
    im_band2 /= np.std(im_band2)    
    im = np.concatenate([im_band1, im_band2], axis=2)
    X.append(im)
    y.append(label)    
X = np.array(X)
y = np.array(y)
print 'X.shape:', X.shape
print 'y.shape:', y.shape

X.shape: (1604, 75, 75, 2)
y.shape: (1604,)


In [3]:
# # Test
# df = pd.read_json('./data/test.json')
# X_test, y_test = [], []
# for im_band1, im_band2 in zip(df['band_1'], df['band_2']):
#     im_band1 = np.array(im_band1).reshape(75, 75, 1)
#     im_band2 = np.array(im_band2).reshape(75, 75, 1)
#     im_band1 /= np.std(im_band1)
#     im_band2 /= np.std(im_band2)    
#     im = np.concatenate([im_band1, im_band2], axis=2)
#     X_test.append(im)
#     y_test.append(label)    
# X_test = np.array(X_test)
# y_test = np.array(y_test)
# print 'X_test.shape:', X_test.shape
# print 'y_test.shape:', y_test.shape

## Train - Val Split

In [4]:
N_SPLITS = 5
MODEL_NUMBER = 5
skf = StratifiedKFold(n_splits=N_SPLITS, random_state=RANDOM_SEED, shuffle=True)
cv = list(skf.split(X, y))

In [5]:
train_i, val_i = cv[MODEL_NUMBER - 1]
X_train, y_train = X[train_i], y[train_i]
X_val, y_val = X[val_i], y[val_i]
print 'X_train.shape:', X_train.shape
print 'y_train.shape:', y_train.shape
print 'X_val.shape:', X_val.shape
print 'y_val.shape:', y_val.shape
print 'np.mean(y_train):', np.mean(y_train)
print 'np.mean(y_val):', np.mean(y_val)

X_train.shape: (1284, 75, 75, 2)
y_train.shape: (1284,)
X_val.shape: (320, 75, 75, 2)
y_val.shape: (320,)
np.mean(y_train): 0.469626168224
np.mean(y_val): 0.46875


In [6]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
# print 'X_train.shape:', X_train.shape
# print 'X_val.shape:', X_val.shape
# print 'y_train.shape:', y_train.shape
# print 'y_val.shape:', y_val.shape

## Data Augmentation

In [2]:
def bypass(x):
    return x

def h_flip(x):
    return x[:, :, ::-1, :]

def v_flip(x):
    return x[:, ::-1, :, :]

def hv_flip(x):
    return h_flip(v_flip(x))

def rot90(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 90), axis=0) for im in x], axis=0)

def rot180(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 180), axis=0) for im in x], axis=0)

def rot270(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 270), axis=0) for im in x], axis=0)

def rot45(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 45, mode='reflect'), axis=0) for im in x], axis=0)

def rot135(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 135, mode='reflect'), axis=0) for im in x], axis=0)

def rot315(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 315, mode='reflect'), axis=0) for im in x], axis=0)

aug_funcs = [bypass, 
             h_flip, v_flip, hv_flip,
             rot90, rot180, rot270]

In [8]:
# Train
X_train = np.concatenate([func(X_train) for func in aug_funcs], axis=0)
y_train = np.concatenate([y_train] * len(aug_funcs))

# Validation
X_val = np.concatenate([func(X_val) for func in aug_funcs], axis=0)
y_val = np.concatenate([y_val] * len(aug_funcs))

# 
print 'X_train.shape:', X_train.shape
print 'y_train.shape:', y_train.shape
print 'X_val.shape:', X_val.shape
print 'y_val.shape:', y_val.shape

X_train.shape: (8988, 75, 75, 2)
y_train.shape: (8988,)
X_val.shape: (2240, 75, 75, 2)
y_val.shape: (2240,)


# Training

In [9]:
def get_model(input_shape):
    # Architecture
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile the model
    loss = losses.binary_crossentropy
    optimizer = optimizers.Adam()
    metrics = ['accuracy']
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    
    #     
    return model

In [10]:
model = get_model(input_shape=(75, 75, 2))

In [11]:
# Callbacks
def get_lr(epoch):
    lr = (np.random.rand() * 1e-3 + 1e-7)
    print 'lr:', lr
    return lr
MODEL_PATH = './models/model4/model' + str(MODEL_NUMBER) + '.h5'
m_q = 'val_loss'
model_path = MODEL_PATH
check_pt = callbacks.ModelCheckpoint(filepath=model_path, monitor=m_q, save_best_only=True, verbose=1)
early_stop = callbacks.EarlyStopping(patience=6, monitor=m_q, verbose=1)
reduce_lr = callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor=m_q, verbose=1)
schedule_lr = callbacks.LearningRateScheduler(get_lr)
callback_list = [check_pt, early_stop, reduce_lr]

model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callback_list, epochs=200)

Train on 8988 samples, validate on 2240 samples
Epoch 1/200
8988/8988 [==============================] - 6s - loss: 0.4930 - acc: 0.7365 - val_loss: 0.3649 - val_acc: 0.8304
Epoch 2/200
8988/8988 [==============================] - 5s - loss: 0.3011 - acc: 0.8665 - val_loss: 0.2574 - val_acc: 0.8705
Epoch 3/200
8988/8988 [==============================] - 5s - loss: 0.2567 - acc: 0.8922 - val_loss: 0.2925 - val_acc: 0.8656
Epoch 4/200
8988/8988 [==============================] - 5s - loss: 0.2188 - acc: 0.9107 - val_loss: 0.2371 - val_acc: 0.9031
Epoch 5/200
8988/8988 [==============================] - 5s - loss: 0.1977 - acc: 0.9173 - val_loss: 0.2424 - val_acc: 0.8920
Epoch 6/200
8988/8988 [==============================] - 5s - loss: 0.1781 - acc: 0.9292 - val_loss: 0.2830 - val_acc: 0.8777
Epoch 7/200
8988/8988 [==============================] - 5s - loss: 0.1715 - acc: 0.9266 - val_loss: 0.2177 - val_acc: 0.9094
Epoch 8/200
8988/8988 [==============================] - 5s - loss: 0.

In [12]:
# # Finetune
# loss = losses.binary_crossentropy
# optimizer = optimizers.SGD(lr=1e-4)
# metrics = ['accuracy']
# model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

In [13]:
# m_q = 'val_loss'
# model_path = MODEL_PATH
# check_pt = callbacks.ModelCheckpoint(filepath=model_path, monitor=m_q, save_best_only=True, verbose=1)
# early_stop = callbacks.EarlyStopping(patience=5, monitor=m_q, verbose=1)
# reduce_lr = callbacks.ReduceLROnPlateau(patience=2, factor=0.33, monitor=m_q, verbose=1)
# callback_list = [check_pt, early_stop, reduce_lr]

# model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callback_list, epochs=200)

# Predict Test

In [3]:
# Load test data
df = pd.read_json('./data/test.json')
X_test, y_test = [], []
for im_band1, im_band2 in zip(df['band_1'], df['band_2']):
    im_band1 = np.array(im_band1).reshape(75, 75, 1)
    im_band2 = np.array(im_band2).reshape(75, 75, 1)    
    # Preprocess - zero mean
    im_band1 -= np.mean(im_band1)
    im_band2 -= np.mean(im_band2)
    # Preprocess - normalize
    im_band1 /= np.std(im_band1)
    im_band2 /= np.std(im_band2)    
    im = np.concatenate([im_band1, im_band2], axis=2)
    X_test.append(im)    
X_test = np.array(X_test)
print 'X_test.shape:', X_test.shape

X_test.shape: (8424, 75, 75, 2)


In [4]:
y_test_p = 0
weights = [0.25, 0.4 / 3, 0.35, 0.4 / 3, 0.4 / 3]
for i, w in zip(range(5), weights):
    print i
    # Load the model
    MODEL_PATH = './models/model4/model' + str(i + 1) + '.h5'
    model = load_model(MODEL_PATH)
    # predict - tta    
    for func in aug_funcs:
        y_test_p += model.predict(func(X_test), verbose=1).flatten() * w
# y_test_p = y_test_p / (len(aug_funcs) * 5.0)
y_test_p = y_test_p / (len(aug_funcs) * sum(weights))

0
8424/8424 [==============================] - 2s 196us/step
1
8424/8424 [==============================] - 2s 246us/step
2
8424/8424 [==============================] - 2s 195us/step
3
8424/8424 [==============================] - 2s 191us/step
4
8424/8424 [==============================] - 2s 190us/step


In [5]:
df_sub = pd.DataFrame()
df_sub['id'] = df['id']
df_sub['is_iceberg'] = y_test_p.flatten()
df_sub.to_csv('./submissions/sub18.csv', index=False)